In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
from random import random
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)

import settings
import helper
import visual

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')

In [3]:
df_meta_train.head(5)

,width,height,slice,originX,originY,originZ,spacingX,spacingY,spacingZ,segmented_lung,segmented_medi
seriesuid,,,,,,,,,,,
689704,350,350,285,-175.384491,-174.658203,-346.053955,0.683594,0.683594,5.0,0.125633,0.922212
680464,350,350,300,-174.658203,-174.658203,1195.953613,0.683594,0.683594,5.0,0.159373,0.947458
651264,350,350,295,-162.311188,-174.658203,-91.186157,0.683594,0.683594,5.0,0.148750,0.901729
401934,360,360,265,-174.000000,-180.000000,-760.400024,0.703125,0.703125,5.0,0.150447,0.946397
389624,400,400,296,-205.662994,-199.804306,-816.500000,0.782000,0.782000,8.0,0.098559,0.955368


In [4]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [5]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
655923,361,361,260,217.869141,242.899414,102.0,13.396484,16.921875,10.0,-167.800446,-180.147461,-322.552246,0.705078,0.705078,5.0,31.0
664042,330,330,296,209.150391,170.800781,24.0,10.312500,12.246094,8.0,-146.677734,-295.677734,-486.500000,0.644531,0.644531,8.0,31.0
672104,344,344,320,196.523438,160.914062,245.0,5.375000,5.375000,5.0,-180.664062,-311.664062,-833.099976,0.671875,0.671875,5.0,31.0
675354,330,330,288,129.873047,213.017578,216.0,7.734375,9.023438,8.0,-175.677734,-288.677734,-156.000000,0.644531,0.644531,8.0,1.0
493047,302,302,325,210.279297,223.255859,110.0,23.593750,21.234375,5.0,-143.800003,-151.000000,-706.099976,0.589844,0.589844,5.0,1.0
671788,350,350,275,113.476562,99.462891,70.0,6.152344,6.835938,5.0,-170.300415,-174.658203,-433.221680,0.683594,0.683594,5.0,31.0
631067,350,350,310,185.253906,160.302734,220.0,10.253906,8.203125,5.0,-148.511566,-174.658203,1233.173218,0.683594,0.683594,5.0,31.0
657258,350,350,309,127.490234,185.937500,184.0,6.835938,7.519531,12.0,-163.100006,-27.000000,-322.040009,0.683594,0.683594,3.0,32.0
686405,350,350,335,65.283203,195.166016,150.0,5.468750,6.835938,5.0,-203.710022,-174.658203,-350.007812,0.683594,0.683594,5.0,1.0


In [6]:
lungs = list(set(df_annotation.index))
print('distinct lungs in annotation:', len(lungs))

distinct lungs in annotation: 1436


# 2. Generate Negative Labels

In [7]:
wtype = 'lung'

In [8]:
IS_GENERATE = True
df_false_positive = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + f'train/submission_{wtype}.csv', index_col=['seriesuid'])
df_false_positive.index = df_false_positive.index.astype('str')
print('false_positive:', df_false_positive.shape, 'distinct lung:', len(set(df_false_positive.index)))

false_positive: (33454, 6) distinct lung: 1000


In [9]:
# df_false_positive = df_false_positive[df_false_positive['probability'] > 0.95]
# print(df_false_positive.shape)

In [10]:
if IS_GENERATE:
    list_fp = []
    for uid in tqdm(lungs):
        if uid not in df_false_positive.index:
            continue
            
        candidates = df_false_positive.loc[[uid]] #dataframe
        labels = df_annotation.loc[[uid]] # dataframe
        meta = df_meta_train.loc[uid] # slice
        
        if len(candidates) > 0:
            for i,c in candidates.iterrows():
                vcoordX, vcoordY, vcoordZ = c.coordX - meta.originX, c.coordY - meta.originY, c.coordZ - meta.originZ
                # print(vcoordX, vcoordY, vcoordZ)

                flag_pass = True
                if len(labels) > 0:
                    for idx, item in labels.iterrows():
                        dist = np.linalg.norm(np.array([vcoordX, vcoordY, vcoordZ]) - np.array([item.vcoordX, item.vcoordY, item.vcoordZ]))
                        diam = np.linalg.norm(np.array([item.diameterX, item.diameterY, item.diameterZ]))
                        if dist <= settings.CUBE_POS_SIZE + diam:
                            flag_pass = False
                            break

                if flag_pass:
                    fp = {}
                    fp['seriesuid'] = uid
                    fp['width'] = meta.width
                    fp['height'] = meta.height
                    fp['slice'] = meta.slice
                    fp['vcoordX'] = vcoordX
                    fp['vcoordY'] = vcoordY
                    fp['vcoordZ'] = vcoordZ
                    list_fp.append(fp)
                

100%|██████████| 1436/1436 [00:46<00:00, 30.56it/s]


In [11]:
if IS_GENERATE:
    df_fp = pd.DataFrame(list_fp, columns=['seriesuid','width','height','slice','vcoordX','vcoordY','vcoordZ'])
    df_fp = df_fp.set_index('seriesuid')
    df_fp.index = df_fp.index.astype('str')
    df_fp['width'] = df_fp['width'].astype('int')
    df_fp['height'] = df_fp['height'].astype('int')
    df_fp['slice'] = df_fp['slice'].astype('int')
    
    print('total:', len(df_fp), 'lung:', len(set(df_fp.index)))
    
    df_fp.to_csv(settings.PREPROCESS_FALSE_POSITIVE_LUNG_FILE, encoding='utf-8')
    

total: 8064 lung: 841


In [12]:
wtype = 'medi'

In [13]:
IS_GENERATE = True
df_false_positive = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + f'train/submission_{wtype}.csv', index_col=['seriesuid'])
df_false_positive.index = df_false_positive.index.astype('str')
print('false_positive:', df_false_positive.shape, 'distinct lung:', len(set(df_false_positive.index)))

false_positive: (33217, 6) distinct lung: 1000


In [14]:
# df_false_positive = df_false_positive[df_false_positive['probability'] > 0.95]
# print(df_false_positive.shape)

In [15]:
if IS_GENERATE:
    list_fp = []
    for uid in tqdm(lungs):
        if uid not in df_false_positive.index:
            continue
            
        candidates = df_false_positive.loc[[uid]] #dataframe
        labels = df_annotation.loc[[uid]] # dataframe
        meta = df_meta_train.loc[uid] # slice
        
        if len(candidates) > 0:
            for i,c in candidates.iterrows():
                vcoordX, vcoordY, vcoordZ = c.coordX - meta.originX, c.coordY - meta.originY, c.coordZ - meta.originZ
                # print(vcoordX, vcoordY, vcoordZ)

                flag_pass = True
                if len(labels) > 0:
                    for idx, item in labels.iterrows():
                        dist = np.linalg.norm(np.array([vcoordX, vcoordY, vcoordZ]) - np.array([item.vcoordX, item.vcoordY, item.vcoordZ]))
                        diam = np.linalg.norm(np.array([item.diameterX, item.diameterY, item.diameterZ]))
                        if dist <= settings.CUBE_POS_SIZE + diam:
                            flag_pass = False
                            break

                if flag_pass:
                    fp = {}
                    fp['seriesuid'] = uid
                    fp['width'] = meta.width
                    fp['height'] = meta.height
                    fp['slice'] = meta.slice
                    fp['vcoordX'] = vcoordX
                    fp['vcoordY'] = vcoordY
                    fp['vcoordZ'] = vcoordZ
                    list_fp.append(fp)
                

100%|██████████| 1436/1436 [00:50<00:00, 19.67it/s]


In [16]:
if IS_GENERATE:
    df_fp = pd.DataFrame(list_fp, columns=['seriesuid','width','height','slice','vcoordX','vcoordY','vcoordZ'])
    df_fp = df_fp.set_index('seriesuid')
    df_fp.index = df_fp.index.astype('str')
    df_fp['width'] = df_fp['width'].astype('int')
    df_fp['height'] = df_fp['height'].astype('int')
    df_fp['slice'] = df_fp['slice'].astype('int')
    
    print('total:', len(df_fp), 'lung:', len(set(df_fp.index)))
    
    df_fp.to_csv(settings.PREPROCESS_FALSE_POSITIVE_MEDI_FILE, encoding='utf-8')
    

total: 11935 lung: 907


# 3. False Positive Samples Extraction

In [17]:
IS_EXTRACTION = True
if not os.path.exists(settings.PREPROCESS_FPOS_DIR):
    os.mkdir(settings.PREPROCESS_FPOS_DIR)
    os.mkdir(settings.PREPROCESS_FPOS_DIR + 'lung/')
    os.mkdir(settings.PREPROCESS_FPOS_DIR + 'medi/')
    
df_false_positive = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_FALSE_POSITIVE_LUNG_FILE, index_col=['seriesuid'])
df_false_positive.index = df_false_positive.index.astype('str')
print('false_positive:', df_false_positive.shape, 'distinct lung:', len(set(df_false_positive.index)))

false_positive: (8064, 6) distinct lung: 841


In [ ]:
if IS_EXTRACTION:
    for uid in tqdm(lungs):
        if uid not in df_false_positive.index:
            continue
            
        labels = df_false_positive.loc[[uid]]

        lung_l, mask_l = helper.load_lung_array(uid, int(labels['width'].values[0]), int(labels['height'].values[0]), int(labels['slice'].values[0]), wtype='lung')
#         lung_m, mask_m = helper.load_lung_array(uid, int(labels['width'].values[0]), int(labels['height'].values[0]), int(labels['slice'].values[0]), wtype='medi')

        lung_l = lung_l*(mask_l>0)
#         lung_m = lung_m*(mask_m>0)
        
        for idx, item in labels.iterrows():
            lung, wtype = lung_l, 'lung'
            cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, block_size=settings.CUBE_FPOS_SIZE)
            if np.sum(cube) > settings.THRESHOLD_VALID_CUBE:
                helper.save_cube_img(
                        f'{settings.PREPROCESS_FPOS_DIR}{wtype}/{idx}_x{int(item.vcoordX)}_y{int(item.vcoordY)}_z{int(item.vcoordZ)}.png', 
                        cube, rows=8, cols=8)
                
            

 34%|███▍      | 495/1436 [06:11<16:01,  1.02s/it]

In [ ]:
IS_EXTRACTION = True
if not os.path.exists(settings.PREPROCESS_FPOS_DIR):
    os.mkdir(settings.PREPROCESS_FPOS_DIR)
    os.mkdir(settings.PREPROCESS_FPOS_DIR + 'lung/')
    os.mkdir(settings.PREPROCESS_FPOS_DIR + 'medi/')
    
df_false_positive = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_FALSE_POSITIVE_MEDI_FILE, index_col=['seriesuid'])
df_false_positive.index = df_false_positive.index.astype('str')
print('false_positive:', df_false_positive.shape, 'distinct lung:', len(set(df_false_positive.index)))


In [ ]:
if IS_EXTRACTION:
    for uid in tqdm(lungs):
        if uid not in df_false_positive.index:
            continue
            
        labels = df_false_positive.loc[[uid]]

#         lung_l, mask_l = helper.load_lung_array(uid, int(labels['width'].values[0]), int(labels['height'].values[0]), int(labels['slice'].values[0]), wtype='lung')
        lung_m, mask_m = helper.load_lung_array(uid, int(labels['width'].values[0]), int(labels['height'].values[0]), int(labels['slice'].values[0]), wtype='medi')

#         lung_l = lung_l*(mask_l>0)
        lung_m = lung_m*(mask_m>0)
        
        for idx, item in labels.iterrows():              
            lung, wtype = lung_m, 'medi'
            cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, block_size=settings.CUBE_FPOS_SIZE)
            if np.sum(cube) > settings.THRESHOLD_VALID_CUBE:
                helper.save_cube_img(
                        f'{settings.PREPROCESS_FPOS_DIR}{wtype}/{idx}_x{int(item.vcoordX)}_y{int(item.vcoordY)}_z{int(item.vcoordZ)}.png', 
                        cube, rows=8, cols=8)
            

# 3. Validate False Positive Samples

In [ ]:
print(len(glob(settings.PREPROCESS_FPOS_DIR + 'lung/*.png')))
print(len(glob(settings.PREPROCESS_FPOS_DIR + 'medi/*.png')))

In [ ]:
path = '*/'
dataset = glob(settings.PREPROCESS_FPOS_DIR + path + '*.png')
print(len(dataset))

invalid = []
for file in tqdm(dataset):
    cube = helper.load_cube_img(file, rows=8, cols=8)
    
    size_old = settings.CUBE_SIZE
    size_new = 32
    cs = (size_old - size_new) // 2
    ce = (size_old - size_new) // 2 + size_new
    cube = cube[cs:ce, cs:ce, cs:ce]
    
    assert cube.shape == (size_new, size_new, size_new)
    
    if np.sum(cube) <= settings.THRESHOLD_VALID_CUBE // 8:
        invalid.append(file)
        
print(len(invalid))

In [ ]:
for file in invalid:
    os.remove(file)

# 4. Visual Samples By Labels

In [ ]:
EXAMPLE_SERIESUID = '660577'

In [ ]:
for img_file in iglob(settings.PREPROCESS_FPOS_DIR + 'medi/' + f'{EXAMPLE_SERIESUID}*.png'):   
    img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    print(img_file, np.sum(img))
    fig, axs = plt.subplots(1, 1, figsize=(16, 16))
    axs.imshow(img, cmap='gray')
    plt.show()

In [ ]:
for img_file in iglob(settings.PREPROCESS_FPOS_DIR + 'lung/' + f'{EXAMPLE_SERIESUID}*.png'):
    img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    print(img_file, np.sum(img))
    fig, axs = plt.subplots(1, 1, figsize=(16, 16))
    axs.imshow(img, cmap='gray')
    plt.show()